**Task I - simple quantum circuit and operations**



 **Part 1**
*   With 5 qubits 
*   Apply Hadamard operation on every qubit 
*   Apply CNOT operation on (0, 1), (1,2), (2,3), (3,4) 
*   SWAP (0, 4) 
*   Rotate X with pi/2 on any qubit 
*   Plot the circuit 



In [1]:
import torch
import numpy as np
import pennylane as qml

n_qubits = 5 # (4 normal qubits + 1 ancilla)

# here we begin part a), where we create a quantum device for 5 qubits.
dev = qml.device("default.qubit", wires=n_qubits)

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

In [4]:
@qml.qnode(dev, interface="torch")
def pqc_exercise_1(rx_angle):#(q_input_features, q_weights_flat):
    """
    b) Apply Hadamard operation on every qubit 
    c) Apply CNOT operation on (0, 1), (1,2), (2,3), (3,4) 
    d) SWAP (0, 4) 
    e) Rotate X with pi/2 on any qubit 

    """

    # Apply Hadamard operation on every qubit 
    H_layer(n_qubits)

    # Apply CNOT operation on (0, 1), (1,2), (2,3), (3,4) 
    for i in range(0, n_qubits - 1):  
        qml.CNOT(wires=[i, i + 1])

    # SWAP between first and fifth qubit.
    qml.SWAP([0,4])

    # Rotate over X on 0
    qml.RX(rx_angle, wires=0)

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

In [5]:
print(qml.draw(pqc_exercise_1)(rx_angle = np.pi/2))

0: ──H─╭●──────────╭SWAP──RX(1.57)─┤  <Z>
1: ──H─╰X─╭●───────│───────────────┤  <Z>
2: ──H────╰X─╭●────│───────────────┤  <Z>
3: ──H───────╰X─╭●─│───────────────┤  <Z>
4: ──H──────────╰X─╰SWAP───────────┤  <Z>


**Part 2**

In [6]:
dev = qml.device("default.qubit", wires=6)

@qml.qnode(dev, interface="torch")
def pqc_exercise_2(rx_angle):
    """
    a) Apply a Hadmard gate to the first qubit
    b) rotate the second qubit by pi/3 around X
    c) Apply Hadamard gate to the third and fourth qubit
    d) Perform a swap test between the states of the first and second qubit |q1 q2> and the third and fourth qubit |q3 q4>

    """

    # Reshape weights
    # q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Apply Hadamard operation on the first qubit 
    qml.Hadamard(wires=2)

    # Embed features in the quantum node
    qml.RX(rx_angle, wires=3)

    # Apply Hadamard operation on the third and fourth qubit 
    qml.Hadamard(wires=4)
    qml.Hadamard(wires=5)

    # SWAP test: 1st and 2nd qubit in |+>
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    # SWAP test between |q1 q2> and |q3 q4>
    qml.CSWAP(wires=[0, 2, 3])
    qml.CSWAP(wires=[1, 4, 5])

    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)

    exp = [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))] # expected value of ancilla qubit (0-th qubit)
    
    return tuple(exp)

In [7]:
pqc_exercise_2(np.pi/3)

tensor([0.5000, 1.0000], dtype=torch.float64)

In [8]:
print(qml.draw(pqc_exercise_2)(rx_angle = np.pi/3))

0: ──H────────╭●─────H───────┤  <Z>
1: ──H────────│─────╭●─────H─┤  <Z>
2: ──H────────├SWAP─│────────┤     
3: ──RX(1.05)─╰SWAP─│────────┤     
4: ──H──────────────├SWAP────┤     
5: ──H──────────────╰SWAP────┤     
